# Estudo prático da estrutura das redes neurais artificiais - Regressão

#### Sabrina Otoni da Silva - 2023

In [1]:
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as k

from scikeras.wrappers import KerasRegressor

import time

In [2]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')
base.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
base = base.drop(['dateCrawled', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen', 'name', 'seller', 'offerType'], axis = 1)

In [4]:
base = base.loc[(base.price > 10) & (base.price < 350000)]

In [5]:
base.price.mean()

5916.833945186492

In [6]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359291 entries, 0 to 371527
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   price                359291 non-null  int64 
 1   abtest               359291 non-null  object
 2   vehicleType          325745 non-null  object
 3   yearOfRegistration   359291 non-null  int64 
 4   gearbox              342055 non-null  object
 5   powerPS              359291 non-null  int64 
 6   model                341324 non-null  object
 7   kilometer            359291 non-null  int64 
 8   monthOfRegistration  359291 non-null  int64 
 9   fuelType             329900 non-null  object
 10  brand                359291 non-null  object
 11  notRepairedDamage    293305 non-null  object
dtypes: int64(5), object(7)
memory usage: 35.6+ MB


In [7]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}

base = base.fillna(value = valores)

previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [8]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])], remainder='passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

regressor = Sequential([
        tf.keras.layers.Dense(units=158, activation = 'relu', input_dim=316),
        tf.keras.layers.Dense(units=158, activation = 'relu'),
        tf.keras.layers.Dense(units=1, activation = 'linear')])

regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam',
                  metrics = ['mean_absolute_error'])
regressor.fit(previsores, preco_real, batch_size = 300, epochs = 100)

Epoch 1/100
1198/1198 [==============================] - 3s 2ms/step - loss: 3809.3030 - mean_absolute_error: 3809.3030
Epoch 2/100
1198/1198 [==============================] - 2s 2ms/step - loss: 3345.3384 - mean_absolute_error: 3345.3384
Epoch 3/100
1198/1198 [==============================] - 2s 2ms/step - loss: 3131.0417 - mean_absolute_error: 3131.0417
Epoch 4/100
1198/1198 [==============================] - 3s 2ms/step - loss: 2952.1860 - mean_absolute_error: 2952.1860
Epoch 5/100
1198/1198 [==============================] - 3s 2ms/step - loss: 2850.1372 - mean_absolute_error: 2850.1372
Epoch 6/100
1198/1198 [==============================] - 3s 3ms/step - loss: 2811.2915 - mean_absolute_error: 2811.2915
Epoch 7/100
1198/1198 [==============================] - 3s 2ms/step - loss: 2740.0398 - mean_absolute_error: 2740.0398
Epoch 8/100
1198/1198 [==============================] - 3s 2ms/step - loss: 2687.5300 - mean_absolute_error: 2687.5298
Epoch 9/100
1198/1198 [=================

In [9]:
previsoes = regressor.predict(previsores)
print(preco_real.mean())
print(previsoes.mean())

11228/11228 [==============================] - 16s 1ms/step
5916.833945186492
5310.101


In [11]:
inicio = time.time()

previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])],remainder='passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

def criar_rede():
        k.clear_session()
        regressor = Sequential([
        tf.keras.layers.Dense(units=158, activation = 'relu', input_dim=316),
        tf.keras.layers.Dense(units=158, activation = 'relu'),
        tf.keras.layers.Dense(units=1, activation = 'linear')])
        regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam',
                      metrics = ['mean_absolute_error'])
        return regressor

regressor = KerasRegressor(build_fn = criar_rede,
                           epochs = 100,
                           batch_size = 300)

resultados = cross_val_score(estimator = regressor,
                             X = previsores, y = preco_real,
                             cv = 10, scoring = 'neg_mean_absolute_error')
fim = time.time()
print(resultados.mean())
print(resultados.std())
print(fim - inicio)

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 3s 2ms/step - loss: 3964.6152 - mean_absolute_error: 3964.6152
Epoch 2/100
1078/1078 [==============================] - 3s 3ms/step - loss: 3461.1951 - mean_absolute_error: 3461.1951
Epoch 3/100
1078/1078 [==============================] - 2s 2ms/step - loss: 3336.5901 - mean_absolute_error: 3336.5901
Epoch 4/100
1078/1078 [==============================] - 2s 2ms/step - loss: 3078.8499 - mean_absolute_error: 3078.8499
Epoch 5/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2918.7178 - mean_absolute_error: 2918.7178
Epoch 6/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2825.0698 - mean_absolute_error: 2825.0698
Epoch 7/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2767.1592 - mean_absolute_error: 2767.1589
Epoch 8/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2702.6758 - mean_absolute_error: 2702.6758
Epoch 9/100
1078/1078 [=================

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 3s 2ms/step - loss: 3892.2466 - mean_absolute_error: 3892.2466
Epoch 2/100
1078/1078 [==============================] - 2s 2ms/step - loss: 3398.8726 - mean_absolute_error: 3398.8726
Epoch 3/100
1078/1078 [==============================] - 2s 2ms/step - loss: 3128.7473 - mean_absolute_error: 3128.7473
Epoch 4/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2933.0420 - mean_absolute_error: 2933.0420
Epoch 5/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2835.1206 - mean_absolute_error: 2835.1206
Epoch 6/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2773.5815 - mean_absolute_error: 2773.5815
Epoch 7/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2727.8291 - mean_absolute_error: 2727.8289
Epoch 8/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2692.7224 - mean_absolute_error: 2692.7224
Epoch 9/100
1078/1078 [=================

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/100
1078/1078 [==============================] - 4s 3ms/step - loss: 3911.6448 - mean_absolute_error: 3911.6448
Epoch 2/100
1078/1078 [==============================] - 3s 2ms/step - loss: 3377.3420 - mean_absolute_error: 3377.3420
Epoch 3/100
1078/1078 [==============================] - 3s 2ms/step - loss: 3094.1702 - mean_absolute_error: 3094.1702
Epoch 4/100
1078/1078 [==============================] - 3s 2ms/step - loss: 2938.3555 - mean_absolute_error: 2938.3555
Epoch 5/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2851.5044 - mean_absolute_error: 2851.5044
Epoch 6/100
1078/1078 [==============================] - 3s 2ms/step - loss: 2791.3577 - mean_absolute_error: 2791.3577
Epoch 7/100
1078/1078 [==============================] - 3s 2ms/step - loss: 2752.6487 - mean_absolute_error: 2752.6487
Epoch 8/100
1078/1078 [==============================] - 3s 2ms/step - loss: 2727.3086 - mean_absolute_error: 2727.3083
Epoch 9/100
1078/1078 [=================

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 5s 4ms/step - loss: 3910.8367 - mean_absolute_error: 3910.8367
Epoch 2/100
1078/1078 [==============================] - 4s 4ms/step - loss: 3459.1577 - mean_absolute_error: 3459.1577
Epoch 3/100
1078/1078 [==============================] - 4s 4ms/step - loss: 3302.1672 - mean_absolute_error: 3302.1667
Epoch 4/100
1078/1078 [==============================] - 4s 4ms/step - loss: 3040.9250 - mean_absolute_error: 3040.9250
Epoch 5/100
1078/1078 [==============================] - 4s 3ms/step - loss: 2880.7449 - mean_absolute_error: 2880.7449
Epoch 6/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2795.9333 - mean_absolute_error: 2795.9333
Epoch 7/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2754.8003 - mean_absolute_error: 2754.8003
Epoch 8/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2713.5667 - mean_absolute_error: 2713.5667
Epoch 9/100
1078/1078 [=================

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 3s 2ms/step - loss: 3935.3083 - mean_absolute_error: 3935.3083
Epoch 2/100
1078/1078 [==============================] - 2s 2ms/step - loss: 3353.0144 - mean_absolute_error: 3353.0144
Epoch 3/100
1078/1078 [==============================] - 2s 2ms/step - loss: 3114.4888 - mean_absolute_error: 3114.4888
Epoch 4/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2970.0405 - mean_absolute_error: 2970.0405
Epoch 5/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2861.2161 - mean_absolute_error: 2861.2161
Epoch 6/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2802.0408 - mean_absolute_error: 2802.0408
Epoch 7/100
1078/1078 [==============================] - 3s 2ms/step - loss: 2745.2080 - mean_absolute_error: 2745.2080
Epoch 8/100
1078/1078 [==============================] - 2s 2ms/step - loss: 2713.4500 - mean_absolute_error: 2713.4500
Epoch 9/100
1078/1078 [=================

In [11]:
def criar_rede2(loss='mean_squared_error'):
    k.clear_session()
    regressor = Sequential()
    regressor.add(Dense(units = 158, activation = 'relu', kernel_initializer = 'random_uniform', input_dim = 316))
    regressor.add(Dense(units = 158, activation = 'relu', kernel_initializer = 'random_uniform'))
    regressor.add(Dense(units = 1, activation = 'linear'))
    regressor.compile(loss=loss, optimizer='adam', metrics=['mean_absolute_error'])
    return regressor

regressor = KerasRegressor(build_fn = criar_rede2, epochs = 100, batch_size = 300)

parametros = {'loss': ['mean_squared_error', 'mean_absolute_error',
                       'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
                       'squared_hinge']}

grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parametros,                           
                           cv = 10)
grid_search = grid_search.fit(previsores, preco_real)
grid_search.best_params_

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 3s 3ms/step - loss: 64002468.0000 - mean_absolute_error: 4167.9346
Epoch 2/100
1078/1078 [==============================] - 2s 2ms/step - loss: 49199528.0000 - mean_absolute_error: 3559.3921
Epoch 3/100
1078/1078 [==============================] - 2s 2ms/step - loss: 41503968.0000 - mean_absolute_error: 3191.9109
Epoch 4/100
1078/1078 [==============================] - 2s 2ms/step - loss: 37845796.0000 - mean_absolute_error: 2976.4653
Epoch 5/100
1078/1078 [==============================] - 2s 2ms/step - loss: 36582576.0000 - mean_absolute_error: 2924.6924
Epoch 6/100
1078/1078 [==============================] - 2s 2ms/step - loss: 35809756.0000 - mean_absolute_error: 2876.4219
Epoch 7/100
1078/1078 [==============================] - 2s 2ms/step - loss: 35099804.0000 - mean_absolute_error: 2833.7839
Epoch 8/100
1078/1078 [==============================] - 2s 2ms/step - loss: 34600848.0000 - mean_absolute_error: 2796.3621
Epoch 9/

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 3s 2ms/step - loss: 64383816.0000 - mean_absolute_error: 4178.5884
Epoch 2/100
1078/1078 [==============================] - 2s 2ms/step - loss: 49904144.0000 - mean_absolute_error: 3636.2834
Epoch 3/100
1078/1078 [==============================] - 2s 2ms/step - loss: 43030344.0000 - mean_absolute_error: 3378.9470
Epoch 4/100
1078/1078 [==============================] - 2s 2ms/step - loss: 39337204.0000 - mean_absolute_error: 3149.9675
Epoch 5/100
1078/1078 [==============================] - 2s 2ms/step - loss: 37554892.0000 - mean_absolute_error: 3026.1262
Epoch 6/100
1078/1078 [==============================] - 2s 2ms/step - loss: 36380936.0000 - mean_absolute_error: 2937.5806
Epoch 7/100
1078/1078 [==============================] - 2s 2ms/step - loss: 35704660.0000 - mean_absolute_error: 2896.3037
Epoch 8/100
1078/1078 [==============================] - 2s 2ms/step - loss: 35137924.0000 - mean_absolute_error: 2849.4114
Epoch 9/

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 3s 2ms/step - loss: 64612884.0000 - mean_absolute_error: 4209.6406
Epoch 2/100
1078/1078 [==============================] - 3s 2ms/step - loss: 48095292.0000 - mean_absolute_error: 3532.4346
Epoch 3/100
1078/1078 [==============================] - 3s 3ms/step - loss: 41247908.0000 - mean_absolute_error: 3213.1812
Epoch 4/100
1078/1078 [==============================] - 3s 2ms/step - loss: 38328116.0000 - mean_absolute_error: 3034.0981
Epoch 5/100
1078/1078 [==============================] - 3s 2ms/step - loss: 36905624.0000 - mean_absolute_error: 2935.3811
Epoch 6/100
1078/1078 [==============================] - 3s 2ms/step - loss: 36368100.0000 - mean_absolute_error: 2907.8513
Epoch 7/100
1078/1078 [==============================] - 3s 3ms/step - loss: 35605908.0000 - mean_absolute_error: 2853.9270
Epoch 8/100
1078/1078 [==============================] - 3s 2ms/step - loss: 35040716.0000 - mean_absolute_error: 2812.1724
Epoch 9/

c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\012728631\AppData\Local\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error inste

Epoch 1/100
1198/1198 [==============================] - 5s 3ms/step - loss: 63883536.0000 - mean_absolute_error: 4189.5981
Epoch 2/100
1198/1198 [==============================] - 5s 4ms/step - loss: 46608396.0000 - mean_absolute_error: 3338.8223
Epoch 3/100
1198/1198 [==============================] - 4s 3ms/step - loss: 38634392.0000 - mean_absolute_error: 3014.9739
Epoch 4/100
1198/1198 [==============================] - 5s 4ms/step - loss: 36970376.0000 - mean_absolute_error: 2962.4443
Epoch 5/100
1198/1198 [==============================] - 5s 4ms/step - loss: 35987004.0000 - mean_absolute_error: 2913.2251
Epoch 6/100
1198/1198 [==============================] - 4s 4ms/step - loss: 35100072.0000 - mean_absolute_error: 2852.2590
Epoch 7/100
1198/1198 [==============================] - 5s 4ms/step - loss: 34787940.0000 - mean_absolute_error: 2841.7368
Epoch 8/100
1198/1198 [==============================] - 4s 3ms/step - loss: 33836088.0000 - mean_absolute_error: 2769.3965
Epoch 9/

{'loss': 'mean_squared_error'}